In [1]:
%autosave 10

Autosaving every 10 seconds


In [3]:
#Testing the autosave function

In [5]:
#Now this is testing the automatic github pushing, is this working?

In [7]:
#Yes it is! Yay!

In [9]:
#Restarted the kernel, does it work now?

In [ ]:
#It does, into bash, you have to type the following: "python auto_commit.py"

### Continuing ML Tutorials and learning how to autosave on Github

### (That is what the above functions are showing)

# Now, we will do Neural Network for Cancer Prediction

## This tutorial is devised from Vappiah's Bioinformatics Projects for learning ML

### We will use RNA-seq data that was processed for differential gene expression in cancer types. The data we use is from https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq and will be a tabular data set

### The rows are cancer samples and the columns and gene counts, the last column is cancer categories

In [28]:
#We need the following libraries
import pandas as pd
import numpy as np

In [32]:
#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
#Preprocessing the data before ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [38]:
#For classification of the ML model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [44]:
#Let's read our data! (here, we use it from a github repo)
file_url='https://github.com/vappiah/Machine-Learning-Tutorials/raw/main/data/cancer_gene_expression.zip'
dataframe=pd.read_csv(file_url)

In [ ]:
#Now, let's explore the data we will study and clean it

In [45]:
print(dataframe.shape)

(801, 8001)


In [48]:
#To check columns 1-3
print(dataframe.columns[0:3])

Index(['gene_1', 'gene_2', 'gene_3'], dtype='object')


In [50]:
#To check last column
dataframe.columns[-1]

'Cancer_Type'

In [52]:
#What data is missing?
datanul=dataframe.isnull().sum()
g=[i for i in datanul if i>0]

print('columns with missing values:%d'%len(g))

columns with missing values:0


In [54]:
#How many cancer types, or in our purposes, classes?
print(dataframe['Cancer_Type'].value_counts())

Cancer_Type
BRCA    300
KIRC    146
LUAD    141
PRAD    136
COAD     78
Name: count, dtype: int64


In [58]:
#Preprocess data 
X=dataframe.iloc[:,0:-1]
y=dataframe.iloc[:,-1]